In [1]:
import os
import numpy as np
from PIL import Image, ImageFile
from skimage import io
import matplotlib.pyplot as plt
import psycopg2
import tifffile
from scipy import misc
from tifffile import TiffFile
from tifffile.tifffile import TIFF_DECOMPESSORS
from configparser import ConfigParser
from batch_tiff_to_jpeg import tiff_to_jpg
from psycopg2.extras import DictCursor
from visualization import get_connection
import progressbar

In [2]:
conn = get_connection()
cur = conn.cursor()
lag_interval_min = 30

# for each position, select first time and last time car was in this position
cur.execute("SELECT DISTINCT COUNT(camera_pos) AS count FROM snapshots")

position_count = cur.fetchone()['count']
cur = conn.cursor('server_cursor')
print(cur.itersize)
cur.execute("""SELECT \
        ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as position, \
    camera_pos, min(timestamp) AS minimum, max(timestamp) AS maximum, \
    min(imagepath) \
    FROM snapshots \
    GROUP BY camera_pos""")
# somehow, grouping by proj matrix shows me same images, but grouping by camera pos does not
cur.execute("""SELECT \
        min(ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)]) as position, \
    min(camera_pos), min(timestamp) AS minimum, max(timestamp) AS maximum, \
    min(imagepath) \
    FROM snapshots \
    GROUP BY proj_matrix""")

positions = {}
print('positions_total: {}'.format(position_count))

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ', progressbar.FileTransferSpeed()]
pbar = progressbar.ProgressBar(widgets=widgets, max_value = position_count).start()

for i, row in enumerate(cur):
    pbar.update(i)
    positions[tuple(row['position'])] = {
        'min': row['minimum'],
        'max': row['maximum']
    }
pbar.finish()

timespans = {key:pos['max'] - pos['min'] for key, pos in positions.items()}
timespans


N/A% 0 |                                                            |   0.0 s/B

2000
positions_total: 291604


  1% 3109 |                                                       |   2.2 KiB/s

[[754.780151367188, 0.141085624694824, 64.4106979370117], '0101000080000000C03D96874000000000180FC23F000000E0481A5040', datetime.datetime(2017, 11, 25, 18, 0, 42, 471531, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), datetime.datetime(2017, 11, 25, 18, 0, 42, 471531, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), '2017-11-25--18-00-42--471']


100% 291604 |#####################################################|  28.2 KiB/s


{(793.452880859375,
  0.215388298034668,
  64.6162567138672): datetime.timedelta(0),
 (2532.501953125, 0.457237720489502, 94.999153137207): datetime.timedelta(0),
 (755.067077636719, 0.87470531463623, 64.460807800293): datetime.timedelta(0),
 (793.876770019531, 0.89374828338623, 64.6605682373047): datetime.timedelta(0),
 (0.273797988891602,
  476.963073730469,
  147.213760375977): datetime.timedelta(0),
 (1.19290590286255, 476.533660888672, 147.230224609375): datetime.timedelta(0),
 (0.0269937515258789,
  1437.49353027344,
  276.532196044922): datetime.timedelta(0),
 (0.104589462280273,
  622.950622558594,
  208.219589233398): datetime.timedelta(0),
 (0.0537419319152832,
  1745.80651855469,
  221.69873046875): datetime.timedelta(0),
 (0.115151882171631,
  6641.59814453125,
  32.7650108337402): datetime.timedelta(0),
 (0.425414323806763,
  1745.29528808594,
  221.778671264648): datetime.timedelta(0),
 (1.30630540847778, 623.366760253906, 208.21940612793): datetime.timedelta(0),
 (0.5849